In [293]:
import pandas as pd
import numpy as np
import os, time, random

#### Identify Directories

#### Web Scraping Lat/Longitudes (w/ Selenium)

In [294]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib

# https://pythonprogramming.net/urllib-tutorial-python-3/
# https://www.linkedin.com/pulse/how-easy-scraping-data-from-linkedin-profiles-david-craven/

In [295]:
'''
driver_path = "/Users/Michael/chromedriver"
driver = webdriver.Chrome(driver_path)

soup.find("input",id="latitude")
soup.find("input",id="longitude")

Australia bounding box: ('Australia', (113.338953078, -43.6345972634, 153.569469029, -10.6681857235))
found here: https://gist.github.com/graydon/11198540

coordinates = [(-33.86,151.2),
               (113.338953078, -43.6345972634),
               (153.569469029, -10.6681857235)
               
              
              ]


'''

'\ndriver_path = "/Users/Michael/chromedriver"\ndriver = webdriver.Chrome(driver_path)\n\nsoup.find("input",id="latitude")\nsoup.find("input",id="longitude")\n\nAustralia bounding box: (\'Australia\', (113.338953078, -43.6345972634, 153.569469029, -10.6681857235))\nfound here: https://gist.github.com/graydon/11198540\n\ncoordinates = [(-33.86,151.2),\n               (113.338953078, -43.6345972634),\n               (153.569469029, -10.6681857235)\n               \n              \n              ]\n\n\n'

In [296]:
'''
# coordinates = (lat,long)
coordinates = [(-33.86,151.2)]

driver_path = "/Users/Michael/chromedriver"
driver = webdriver.Chrome(driver_path)

url = "https://www.latlong.net/Show-Latitude-Longitude.html"
driver.get(url)

#time.sleep(3)

source = driver.page_source
soup = BeautifulSoup(source,"html")

input_lat = driver.find_element_by_id("latitude")
input_long = driver.find_element_by_id("longitude")

input_lat.send_keys(str(lat))
input_long.send_keys(str(long))

convert = driver.find_element_by_class_name("button")
convert.click()

result_source = driver.page_source
result_soup = BeautifulSoup(result_source,"html")

location = driver.find_element_by_class_name("bgw").text

#time.sleep(3)
driver.close()
'''


'\n# coordinates = (lat,long)\ncoordinates = [(-33.86,151.2)]\n\ndriver_path = "/Users/Michael/chromedriver"\ndriver = webdriver.Chrome(driver_path)\n\nurl = "https://www.latlong.net/Show-Latitude-Longitude.html"\ndriver.get(url)\n\n#time.sleep(3)\n\nsource = driver.page_source\nsoup = BeautifulSoup(source,"html")\n\ninput_lat = driver.find_element_by_id("latitude")\ninput_long = driver.find_element_by_id("longitude")\n\ninput_lat.send_keys(str(lat))\ninput_long.send_keys(str(long))\n\nconvert = driver.find_element_by_class_name("button")\nconvert.click()\n\nresult_source = driver.page_source\nresult_soup = BeautifulSoup(result_source,"html")\n\nlocation = driver.find_element_by_class_name("bgw").text\n\n#time.sleep(3)\ndriver.close()\n'

In [366]:
t0 = time.time()

# instantiate driver
driver_path = "/Users/Michael/chromedriver"
driver = webdriver.Chrome(driver_path)

# open chrome, go to URL
url = "https://www.latlong.net/Show-Latitude-Longitude.html"
driver.get(url)

# iterate through lat/long values, submit request, save location string result

N = 2000
if location_list == None:
    location_list = []
else: pass

#location_list = []
for num in range(N):
    
    # generate random lats + longs (must be inside Australia bounding box)
    lat = random.randint(-43,-10) + random.randint(0,100)/100
    long = random.randint(113,153) + random.randint(0,100)/100

    # find lat + long input elements
    input_lat = driver.find_element_by_id("latitude")
    input_long = driver.find_element_by_id("longitude")

    # clear input elements
    input_lat.clear()
    input_long.clear()
    
    # input a lat + long pair (must be in string format)
    input_lat.send_keys(str(lat))
    input_long.send_keys(str(long))

    # convert input lat + long to address
    convert = driver.find_element_by_class_name("button")
    convert.click()

    # record request output
    location = driver.find_element_by_class_name("bgw").text
    
    # if no address is found, record 'None'
    if len(location)==0:
        location = "None"
    
    # append lat + long + location to list
    location_list.append((lat,long,location))
    
    if (num+1) % 100 == 0:
        print(num+1)

# close browser
driver.close()

t1 = time.time()

100
200
300
400
500
600
700
800
900
1000


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=78.0.3904.108)


In [367]:
len(location_list)

6603

In [368]:
mins = (t1-t0)/60

print("No of minutes:",mins)

# approv 38.5 seconds for 10 sets of coordinates
# approx 64 minutes per 1000 sets of coordinates

No of minutes: -737.4036483367285


#### Extracting States from Location Strings

In [369]:
states = ['Western Australia','South Australia','New South Wales',
          'Tasmania','Northern Territory','Queensland','Victoria']

In [370]:

result = []
for lat,long,location in location_list:
    location = location.split('\n')[0]
    for state in states:
        if state.lower() in location.lower():
            res = (lat,long,state)
            break
        else:
            res = (lat,long,'None')

        
    result.append(res)
        
print(len(location_list))
print(len(result))


6603
6603


In [371]:
df = pd.DataFrame(result,columns=['Lat','Long','State'])
print(df.shape)

(6603, 3)


In [376]:
df.tail()

,Lat,Long,State
6598,-38.88,149.83,None
6599,-28.13,130.67,South Australia
6600,-13.54,124.60,None
6601,-32.35,150.38,New South Wales
6602,-16.11,124.97,Western Australia


In [373]:
df['State'].value_counts()

None                  3404
Western Australia      952
Queensland             704
Northern Territory     586
South Australia        526
New South Wales        313
Victoria                83
Tasmania                35
Name: State, dtype: int64

In [374]:
'''
k = 12
location_list[:k]
result[:k]
'''

'\nk = 12\nlocation_list[:k]\nresult[:k]\n'

In [375]:
df.to_csv("coordinates_and_states_pull7.csv",index=False)

#### Plotting Data